# Converting the old RENI models into the Nerfstudio format for comparisons.

In [1]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import yaml
from nerfstudio.cameras.rays import RaySamples, Frustums

from reni.configs.reni_config import RENIField
from reni.field_components.field_heads import RENIFieldHeadNames
from reni.data.datamanagers.reni_datamanager import RENIDataManager
from reni.utils.utils import find_nerfstudio_project_root, rot_z
from reni.utils.colourspace import linear_to_sRGB

# setup config
test_mode = 'val'
world_size = 1
local_rank = 0
device = 'cuda:0'

project_root = find_nerfstudio_project_root(Path(os.getcwd()))
# set current working directory to nerfstudio project root
os.chdir(project_root)

In [21]:
ndims = 49

latest_version = f'/workspace/outputs/reni/old_reni_models/latent_dim_{ndims}/nerfstudio_models/step-000001000.ckpt'

example_ckpt = torch.load(latest_version, map_location='cpu')

In [22]:
path_to_old_models = Path('/workspace/outputs/old_RENI/RENIVariationalAutoDecoder/network_128_5')
config_path = path_to_old_models / f'ndims_{ndims}' / 'files' / 'config.yaml'
eval_reni_path = path_to_old_models / f'ndims_{ndims}' / 'files' / 'RENI_Latent.pt'

# load config
config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)

# load checkpoint
ckpt = torch.load(eval_reni_path, map_location='cpu')

In [23]:
print(f'Nerfstudio Model eval_mu shape = {example_ckpt["pipeline"]["_model.field.eval_mu"].shape}')
print(f'Old model eval_mu shape = {ckpt["mu"].shape}')
print(f'Nerfstudio Model eval_logvar shape = {example_ckpt["pipeline"]["_model.field.eval_logvar"].shape}')
print(f'Old model eval_logvar shape = {ckpt["log_var"].shape}')

Nerfstudio Model eval_mu shape = torch.Size([21, 36, 3])
Old model eval_mu shape = torch.Size([21, 49, 3])
Nerfstudio Model eval_logvar shape = torch.Size([21, 36, 3])
Old model eval_logvar shape = torch.Size([21, 49, 3])


In [24]:
# for all keys in exmple_ckpt['pipelines'] that start with _model.field.network
# print the key and the shape of the value
for key, value in example_ckpt['pipeline'].items():
    if key.startswith('_model.field.network'):
        print(f'{key} shape = {value.shape}')

for key, value in ckpt.items():
    if key.startswith('net'):
        print(f'{key} shape = {value.shape}')

_model.field.network.net.0.linear.weight shape = torch.Size([128, 1370])
_model.field.network.net.0.linear.bias shape = torch.Size([128])
_model.field.network.net.1.linear.weight shape = torch.Size([128, 128])
_model.field.network.net.1.linear.bias shape = torch.Size([128])
_model.field.network.net.2.linear.weight shape = torch.Size([128, 128])
_model.field.network.net.2.linear.bias shape = torch.Size([128])
_model.field.network.net.3.linear.weight shape = torch.Size([128, 128])
_model.field.network.net.3.linear.bias shape = torch.Size([128])
_model.field.network.net.4.linear.weight shape = torch.Size([128, 128])
_model.field.network.net.4.linear.bias shape = torch.Size([128])
_model.field.network.net.5.linear.weight shape = torch.Size([128, 128])
_model.field.network.net.5.linear.bias shape = torch.Size([128])
_model.field.network.net.6.linear.weight shape = torch.Size([3, 128])
_model.field.network.net.6.linear.bias shape = torch.Size([3])
net.0.linear.weight shape = torch.Size([128,

In [25]:
# update the nerfstudio model with the old model
example_ckpt['pipeline']['_model.field.eval_mu'] = ckpt['mu']
example_ckpt['pipeline']['_model.field.eval_logvar'] = ckpt['log_var']
# update the network weights
for key, value in ckpt.items():
    if key.startswith('net'):
        example_ckpt['pipeline']['_model.field.network.' + key] = value

In [26]:
# overwrite the nerfstudio model with the updated weights
torch.save(example_ckpt, latest_version)

In [47]:
example_ckpt

{'step': 8000,
 'pipeline': OrderedDict([('_model.device_indicator_param', tensor([])),
              ('_model.field.train_mu',
               tensor([[[ 0.4504, -0.4230,  0.1640],
                        [ 0.1191, -1.2989,  0.3765],
                        [ 0.6542, -0.1845,  0.9187],
                        ...,
                        [-0.3809,  0.4449,  0.0341],
                        [ 0.4024, -0.3285, -1.4954],
                        [ 0.8239, -1.9142,  1.1930]],
               
                       [[ 0.0427,  0.2233, -0.1915],
                        [ 1.6666,  0.1397, -0.7270],
                        [-0.5604,  0.7917,  0.6892],
                        ...,
                        [ 0.4140,  0.1671,  0.1032],
                        [-0.7822, -0.0294, -0.3978],
                        [ 0.0231, -0.3737,  0.6771]],
               
                       [[ 0.2821, -0.5492, -1.6808],
                        [ 0.3394,  0.5122, -0.6496],
                        [-0.1790, -1.4